# Лекция 9. Последние штрихи

* Исключения
* Декораторы
* Управление атрибутами

# Исключения

Мы уже знаем, как перехватывать исключения

In [31]:
try:    
    a = 1 / 0
    assert a > 0
    
except (ZeroDivisionError, AssertionError): # можно перечислить несколько
    print("Invalid operation")
else:
    # а это будет выполнено только если исключения внутри try не возникли
    print("ELSE")
finally:
    # этот блок будет всегда выполнен
    print("FINALLY")

Invalid operation
FINALLY


Генерировать исключения тоже легко. Это можно сделать с помощью оператора `raise`

> `raise <ExceptionClass>` - данный оператор создает исключение, которое может быть перехвачено обработчиком, написанным программистом. В противном случае, данное исключение будет перехвачено стандартным обработчиком и приведет к остановке программы. Если использовать оператор `raise` без аргумента, то будет повторно сгенерировано самое последнее исключение.

In [2]:
try:
    # можно указать класс (будет создан экземпляр)
    raise ZeroDivisionError
    # а можно указать экземпляр, тогда будет возвращен этот экземпляр
    raise ZeroDivisionError()
except (ZeroDivisionError, AssertionError) as e:
    print("Invalid operation", e)

Invalid operation 


In [3]:
try:
    raise ZeroDivisionError
except ZeroDivisionError:
    print("Got ZeroDivisionError")
    raise

Got ZeroDivisionError


ZeroDivisionError: 

Также есть возможность условной генерации исключений с помощью оператора `assert`, который часто используется для отладки на стадии разработки

> `assert <condition> [, <data>]` - позволяет генерировать исключения, если какое-то условие не выполняется. 

In [1]:
# Срабатывает, если не выполнении условия

try:
    # все ок
    assert 5 > 2

    # все плохо
    assert 5 < 2, 'FAIL'
    
    # Эквивалентно
    if not(5 < 2):
        raise AssertionError("FAIL")
        
except AssertionError as e:
    print("ASSERT:", e)

ASSERT: ['FAIL', 1]


Есть также универсальный класс исключений **Exception**

In [2]:
try:
    raise Exception("item", 1, 3)
        
except Exception as e:
    print(e)
    print(e.args[0], e.args[1])

('item', 1, 3)
item 1


## Создание своих исключений

Выше мы использовали встроенные исключения. Python также позволяет программисту создавать собственные уникальные исключения.

In [206]:
# Нам нужно просто отнаследоваться от базового класса для исключений

class MyOwnCrazyException(Exception): pass

In [207]:
try:
    raise MyOwnCrazyException("FAIL")
except MyOwnCrazyException as e:
    print("My Exception")
    print(e.args)

My Exception
('FAIL',)


In [19]:
# или вариант посложнее

class MyOwnCrazyException2(Exception):
    def __init__(self, *args, **kargs):
        super().__init__(*args, **kargs)
        
    def __str__(self):
        return "[MyOwnCrazyException2]: " + ", ".join([str(arg) for arg in self.args])
    
    def __repr__(self):
        return self.__str__()

In [20]:
try:
    raise MyOwnCrazyException2("1", "4", [3.4])
except MyOwnCrazyException2 as e:
    print(e)

[MyOwnCrazyException2]: 1, 4, [3.4]


## Иерархия исключений

Исключения образуют иерархию, что позволяет перехватывать исключения по их предку, а не по конкретному типу исключения. 

Все исключения наследуются от `BaseException`. Класс `Exception` также наследуется от этого класса. 

In [57]:
class MyException(Exception): pass


try:
    raise MyException("FAIL")
except Exception as e:
    print(e)

FAIL


Это очень удобно для формирования категорий исключений

In [214]:
class GeneralError(Exception): pass

class IOError(GeneralError): pass
class InvalidOperationError(GeneralError): pass
class ConnectionError(GeneralError): pass

try:
    # Наше исключение
    #raise ConnectionError
    
    # Встроенное
    # raise ZeroDivisionError
    
    # Специальное встроенное
    raise SystemExit
except GeneralError as e:
    print("Custom exception: ", repr(e))
except Exception as e:
    print("Exception: ", repr(e))
except BaseException as e:
    print("BaseException: ", repr(e))
    raise
except:
    print("Unknown exception")

Exception:  ZeroDivisionError()


# Декораторы

> __Декораторы__ - это довольно изящный способ указания дополнительного управляющего или дополняющего кода для функций и классов.
 
Создать декоратор довольно просто - это обычная функция/класс, которая получает исполняемый объект и затем возвращает также исполняемый объект.

In [6]:
# простейший декоратор, который ничего не делает
def decorator(F):
    return F

# простейший декоратор, который ничего не делает
class decorator:
    def __init__(self, F):
        self.F = F
    def __call__(self, *args, **kargs):
        return self.F(*args, **kargs)  

Использовать декоратор также легко, для этого используется символ `@`

In [7]:
@decorator
def my_function(arg):
    print(arg)
    
my_function("hi")

hi


Что эквивалентно

In [85]:
def my_function(arg):
    print(arg)
    
my_function = decorator(my_function)

my_function("hi")

hi


## Примеры

In [4]:
# Декоратор, который умножает аргументы на 2

def x2arg(F):
    def wrapper(*args, **kargs):
        nargs = [2*arg for arg in args]
        nkargs = {k:2*v for k, v in kargs.items()}
        return F(*nargs, **nkargs)
    return wrapper

@x2arg
def Add(a, b):
    return a + b
    
Add(1, 1)

'1111'

In [5]:
# а можно несколько раз

@x2arg
@x2arg
def Add(a, b):
    return a + b

Add(1, 1)

8

In [6]:
class counter:
    """
        Декоратор-класс, который считает вызовы функции
    """
    def __init__(self, F):
        self.F = F
        self.counter = 0
        
    def __call__(self, *args, **kargs):
        self.counter += 1
        return self.F(*args, **kargs)
 
@counter
def Add(a, b):
    return a + b


Add = counter(Add)

print(Add.counter)

Add(1, 1)
print(Add.counter)

Add(1, 1)
print(Add.counter)

Add(1, 1)
print(Add.counter)

Add(1, 1)
print(Add.counter)

0
1
2
3
4


## Декораторы с параметрами

Можно создавать декораторы с параметрами, если помнить, как именно происходит вызов декоратора на самом деле. 

In [ ]:
xArg(Add)

In [7]:
def xArg(mul):
    def xArgDecorator(F):
        def wrapper(*args, **kargs):
            nargs = [mul*arg for arg in args]
            nkargs = {k:mul*v for k, v in kargs.items()}
            return F(*nargs, **nkargs)
        return wrapper
    return xArgDecorator

@xArg(5)
def Add(a, b):
    return a + b

Add(1, 1)

10

Это эквивалентно

In [95]:
def Add(a, b):
    return a + b

Add = xArg(5)(Add)
Add(1, 1)

10

## Декораторы классов

Декорировать классы немного хитрее. Пример ошибочной реализации

In [8]:
# ЭТО РАБОТАТЬ АДЕКВАТНО НЕ БУДЕТ

class decorator:
    def __init__(self, cls):
        print("decorator was created")
        self.C = cls
        
    # перехватываем вызов, чтобы создать экземпляр класса
    def __call__(self, *args, **kargs):
        self.instance = self.C(*args, **kargs)
        self.instance.args = args
        return self
    
    def __getattr__(self, attrname):
        return getattr(self.instance, attrname)
  

@decorator
class Test:
    def __init__(self, *arg, **kargs):
        self.X = "Test"
# эквивалентно
# Test = decorator(Test)

x = Test(1, 2, 3)
print("X = ", x.X, x.args)
print()

# а здесь мы перезапишем x, хотя казалось бы, что не должны
y = Test(5, 4, 5)
print("X = ", x.X, x.args)
print("Y = ", y.X, y.args)

decorator was created
X =  Test (1, 2, 3)

X =  Test (5, 4, 5)
Y =  Test (5, 4, 5)


Правильный путь - это создавать каждый раз объект-обертку

In [116]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args, **kargs):
            self.instance = cls(*args, **kargs)
            self.instance.args = args
            
        def __getattr__(self, attrname):
            return getattr(self.instance, attrname)
    return Wrapper


# Здесь мы подменяем один класс другим классом
@decorator
class Test:
    def __init__(self, *arg, **kargs):
        self.X = "Test"
# эквивалентно
# Test = decorator(Test)

x = Test(1, 2, 3)
print("X = ", x.X, x.args)
print()

# Теперь ок
y = Test(5, 4, 5)
print("X = ", x.X, x.args)
print("Y = ", y.X, y.args)

X =  Test (1, 2, 3)

X =  Test (1, 2, 3)
Y =  Test (5, 4, 5)


In [120]:
# А можно сделать так, мы создаем класс-фасад, который 
# принимает готовый экземпляр

class Wrapper:
    def __init__(self, instance):
        self.instance = instance
    def __getattr__(self, attrname):
        return getattr(self.instance, attrname)
    
def decorator(cls):
    def onCall(*args, **kargs):
        instance = cls(*args, **kargs)
        instance.args = args
        return Wrapper(instance)
    return onCall


@decorator
class Test:
    def __init__(self, *arg, **kargs):
        self.X = "Test"
        
x = Test(1, 2, 3)
print("X = ", x.X, x.args)
print()

# Теперь ок
y = Test(5, 4, 5)
print("X = ", x.X, x.args)
print("Y = ", y.X, y.args)

X =  Test (1, 2, 3)

X =  Test (1, 2, 3)
Y =  Test (5, 4, 5)


# Домашняя работа

## Задача 1

Написать декоратор для вывода времени выполнения функции

In [37]:
def timer1(F):
    ...

class timer2:
    ...
    
@timer1
def AnyFunction(t):
    import time
    time.sleep(t)

AnyFunction(1)

@timer2
def AnyFunction(t):
    import time
    time.sleep(t)
    return 5 

AnyFunction(1)

(AnyFunction): It takes 1.00123 seconds
(AnyFunction): It takes 1.00136


5